# LOLA tutorial

Paper link:https://arxiv.org/pdf/1707.08966.pdf
Read the paper before doing this

# 1.Understanding data

Dataset link:https://zenodo.org/record/2603256#.X7VrlGhKiUm
There are 3 files, contains in total 1.2M training events, 400k validation events and 400k test events. We will work with training dataset, whichi is the 1GB.

In [36]:
import pandas
input_filename = "train.h5"
store = pandas.read_hdf(input_filename,'table')

In [37]:
store.info

<bound method DataFrame.info of             E_0        PX_0        PY_0        PZ_0         E_1        PX_1  \
375  474.071136 -250.347031 -223.651962 -334.738098  103.236237  -48.866222   
377  150.504532  120.062393   76.852005  -48.274265   82.257057   63.801739   
378  251.645386   10.427651 -147.573746  203.564880  104.147797   10.718256   
379  451.566132  129.885437  -99.066292 -420.984100  208.410919   59.033958   
380  399.093903 -168.432083  -47.205597 -358.717438  273.691956 -121.926941   
..          ...         ...         ...         ...         ...         ...   
587  206.171997   13.942102  114.328499 -171.001465  231.602356   19.010832   
591  263.984161  -40.649391 -104.321312  239.065552  238.690689    8.786323   
592   61.417538   42.901291   43.947723   -0.436818   45.521763   31.723654   
593  261.215302   12.780115 -132.699203  224.635300  224.066376   52.028233   
596  221.000015  -32.458290 -127.960335 -177.238876  109.126007  -16.046606   

           PY_1    

We can see there are 1211000 rows x 806 columns. 1211000 means there are 1.2 million jets, 806 columns means we have E0,PX0,PY0,PZ0 to E199.PX199,PY199,PZ199(800) with  truthE,truthPX,truthPY,truthPZ,ttv,and is_signal_new.

This means for each jet, we are at most 200 momenta, 1 truth momenta and we can check what kind of dataset by ttv, and wheather it is a signal(top ) or a background(qcd) by is_signal_new(1 for signal 0 for background)


# 2.Data Process

Since the input for LOLA is four momenta array, we need to process them to a four momenta

In [47]:
#first we will split the train dataset from signal and background.
signal = store[store['is_signal_new']==1]
background = store[store['is_signal_new']==0]
print(signal.shape)
print(background.shape)
#we can see there are 600k of each

(605477, 806)
(605523, 806)


In [55]:
def loadmomenta(dataset, nConstituents=40):
    #this function takes a input of top tagging dataset and return a four momenta array
    momenta = dataset.values[:, :nConstituents*4]
    momenta = np.reshape(momenta, (len(momenta), nConstituents, 4))
    momenta = np.transpose(momenta, [0, 2, 1])
    labels = dataset.values[:, -1]
    labels = keras.utils.to_categorical(labels, 2)
    indices = np.random.permutation(len(labels))
    return momenta[indices],labels[indices]

In [ ]:
signal_momenta,signal_labels=loadmomenta(signal)
background_momenta,background_labels=loadmomenta(background)

In [68]:
momenta = np.append(background_momenta, signal_momenta, axis=0)
labels = keras.utils.to_categorical(np.append(background_label, signal_labels), 2)

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

### 3.Model

Cola class and Lola class

In [78]:

import h5py
  
from keras.layers import *
from keras import Model
from keras.models import Sequential
from keras.layers.advanced_activations import PReLU
from keras import regularizers
import numpy as np
import keras, time
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt

class CoLa():
    """Combination layer that returns the input and appends some linear 
    combinations along the last dimension
    """
    def __init__(self, nAdded, **kwargs):
        # Set the number of added combinations
        self.nAdded = nAdded
        super(CoLa, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create trainable weight for the linear combination
        self.combination = self.add_weight(name='combination',
                    shape=(input_shape[2], self.nAdded),
                    initializer='uniform',
                    trainable=True)
        super(CoLa, self).build(input_shape)

    def call(self, x):
        # Generate combinations and return input with appended with combinations
        combined = K.dot(x, self.combination)
        return K.concatenate([x, combined], axis=2)

    def compute_output_shape(self, input_shape):
        self.out_shape = (input_shape[0], 
                    input_shape[1], 
                    input_shape[2] + self.nAdded)
        return self.out_shape

    def get_config(self):
        # Store nAdded value for loading saved models later
        base_config = super(CoLa, self).get_config()
        base_config['nAdded'] = self.nAdded
        return base_config


class LoLa():
    """Lorentz Layer adapted from arXiv:1707.08966
    From an input of 4 vectors generate some physical quantities that serve as
    input to a classifiction network
    """
    def __init__(self, **kwargs):
        super(LoLa, self).__init__(**kwargs)

    def build(self, input_shape):
        initializer = keras.initializers.TruncatedNormal(mean=0., stddev=0.1)
        metric = keras.initializers.Constant(value=[[1., -1., -1., -1.]])
        # Trainable metric for 4-vector multiplication
        self.metric = self.add_weight(name='metric',
                    shape=(1, 4),
                    initializer=metric,
                    trainable=True)

        # Weights for the linear combination of energies
        self.energyCombination = self.add_weight(name='energyCombination',
                    shape=(input_shape[-1], input_shape[-1]),
                    initializer=initializer,
                    trainable=True)
        
        # Weights for the linear combinations of distances
        self.distanceCombination = self.add_weight(name='distanceCombination',
                    shape=(input_shape[2], 4),
                    initializer=initializer,
                    trainable=True)
        super(LoLa, self).build(input_shape)

    def call(self, x):
        def getDistanceMatrix(x):
            """Input:
            x, (batchsize, features, nConst) - array of vectors
            Returns:
            dists, (batchsize, nConst, nConst) - distance array for every jet
            """
            part1 = -2 * K.batch_dot(x, K.permute_dimensions(x, (0, 2, 1)))
            part2 = K.permute_dimensions(K.expand_dims(K.sum(x**2, axis=2)), (0, 2, 1))
            part3 = K.expand_dims(K.sum(x**2, axis=2))
            dists = part1 + part2 + part3
            return dists

        # Get mass of each 4-momentum
        mass = K.dot(self.metric, K.square(x))
        mass = K.permute_dimensions(mass, (1, 0, 2))

        # Get pT of each 4-momentum
        pT = x[:, 1, :] ** 2 + x[:, 2, :] ** 2
        pT = K.sqrt(K.reshape(pT, (K.shape(pT)[0], 1, K.shape(pT)[1])))
        
        # Get a learnable linear combination of the energies of all constituents
        energies = K.dot(x[:, 0, :], self.energyCombination)
        energies = K.reshape(energies, 
                            (K.shape(energies)[0], 1, K.shape(energies)[1]))
 
        # Get the distance matrix and do some linear combination
        dists_3 = getDistanceMatrix(
                            K.permute_dimensions(x[:, 1:, :], (0, 2, 1)))
        dists_0 = getDistanceMatrix(
                            K.permute_dimensions(x[:, 0, None, :], (0, 2, 1)))
        dists = dists_0 - dists_3
        
        dists = K.dot(dists, self.distanceCombination)
        dists = K.permute_dimensions(dists, (0, 2, 1))

        return K.concatenate([mass, pT, energies, dists], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 7, input_shape[2])


This is the full model

In [79]:

class LoLaClassifier(layers):
    def __init__(self, nConstituents, nAdded, tag=0):
        self.tag = tag
        self.model = self._genNetwork(nConstituents, nAdded)
        pass

    def _genNetwork(self, nConstituents, nAdded):
        input = Input((4, nConstituents))
        layer = input

        # Combination layer adds nAdded linear combinations of vectors
        layer = CoLa(nAdded=nAdded, name='cola')(layer)
        print (layer.value)
        # LoLa replaces the 4 vectors by physically more meaningful vectors
        layer = LoLa(name='lola')(layer) 
        print (layer)

        # Connect to a fully connected network for classification
        layer = Flatten()(layer)
        layer = Dense(100, activation='relu')(layer)
        #layer = Dense(50, activation='relu')(layer)
        layer = Dense(10, activation='relu')(layer)
        layer = Dense(2, activation='softmax')(layer)
        
        model = keras.Model(input, layer)
        return model


# Excercise:Try to run the model and train the dataset we created and make a roc curve to see the accuracy.